## МОРСКОЙ БОЙ!!!

Авторы: Дмитрий Преснухин, Владислав Марков 

In [2]:
from tkinter import *  # импорт из библиотеки tkinter
import random as rn


def draw_cell(i, j, status):   # Рисует клетки поля
    font = ['Arial', 15]
    text = ''
    if status == 'Hit':
        text = '⛌'
    if status == 'Kill':
        text = '■'
        font[1] = 80
    if status == 'Miss':
        text = '•'
    cell = Message(battlefield_frame, text=text, relief=RIDGE, font=font, bg='White')
    cell.place(x=i, y=j, width=25, height=25)
    
    
def enter_pressed(event):  # обработка ввода игрока
    global Hint_shown
    inp = commandInput.get().strip().upper()
    commandInput.delete(0, END)
    if inp == '.':  # показыаем / скрываем посказку в зависимости от состояния Hint_shown
        if Hint_shown:
            hidden_battlefield_frame.place_forget()
            Hint_shown = False
        else:
            hidden_battlefield_frame.place(x=300, y=150)
            Hint_shown = True
    else:
        try:
            column = dictionary[inp[0]]
            row = int(inp[1:]) - 1  # -1 т.к. нумерация ячеек в игре начинается с 1, а нумерация элементов списка - с 0
            if field[column][row] == 0 or field[column][row] == 1:
                if row >= 0:
                    x = 25 + 25 * column
                    y = 25 + 25 * row
                    if field[column][row] == 0:
                        field[column][row] = 'Miss'
                        logs_update(f'{inp}: {rn.choice(miss_list)}')
                        logs.itemconfig(END, foreground="red")
                    elif field[column][row] == 1:
                        field[column][row] = 'Processing'
                        kill_check(column, row, inp)
                    draw_cell(x, y, field[column][row])
                else:
                    logs_update('Некорректный ввод!')
                    logs.itemconfig(END, foreground="gray")
            else:
                logs_update('Эта клетка уже открыта(')
                logs.itemconfig(END, foreground="gray")
        except (KeyError, IndexError, ValueError):
            logs_update('Некорректный ввод!')
            logs.itemconfig(END, foreground="gray")
    if ship_1_left.get() == ship_2_left.get() == ship_3_left.get() == ship_4_left.get() == 0:
        logs_update('Вы победили, поздравляем!')
        logs.itemconfig(END, foreground="green")
        commandInput.place_forget()
    print(field)
    
    
def kill_check(column, row, inp):     # Проверяет, убит ли корабль
    field[column][row] = search_area(column, row)
    if field[column][row] == 'Kill':
        draw_killed(column, row)
        count_ships(inp)
    else:
        logs_update(f'{inp}: {rn.choice(hit_list)}')
        
        
def search_area(column, row):       # подфункция функции, проверяющей, убит ли корабль
    for i, j in [(abs(column - 1), row), (column, abs(row - 1)), (column, row + 1), (column + 1, row)]:
        try:
            if field[i][j] == 1:
                return 'Hit'
        except:
            pass
    for i, j in [(abs(column - 1), row), (column, abs(row - 1)), (column, row + 1), (column + 1, row)]:
        try:
            if field[i][j] == 'Hit':
                field[i][j] = 'Processing'
                if search_area(i, j) == 'Hit':
                    field[i][j] = 'Hit'
                    return 'Hit'
                field[i][j] = 'Hit'
        except:
            pass
    return 'Kill'



def draw_killed(column, row):       # Рисуем убитый корабль
    x = 25 + 25 * column
    y = 25 + 25 * row
    draw_cell(x, y, 'Kill')
    global counter
    counter += 1
    draw_misses(column, row)
    for i, j in [(abs(column - 1), row), (column, abs(row - 1)), (column, row + 1), (column + 1, row)]:
        try:
            if field[i][j] == 'Hit':
                field[i][j] = 'Kill'
                draw_misses(i, j)
                draw_killed(i, j)
        except:
            pass

        
def draw_misses(column, row):                 # Рисуем отчки вокруг убитого корабля
    for i, j in [
        (abs(column - 1), row), (column, abs(row - 1)),
        (column, row + 1), (column + 1, row),
        (abs(column - 1), abs(row - 1)), (abs(column - 1), row + 1),
        (column + 1, abs(row - 1)), (column + 1, row + 1)
    ]:                                                               
        try:
            x = 25 + 25 * i
            y = 25 + 25 * j
            if field[i][j] == 0:
                field[i][j] = 'Miss'
                draw_cell(x, y, 'Miss')
        except:
            pass
        

def count_ships(inp):   # Подсчет оставшихся кораблей
    global counter    
    if counter == 1:
        ship_1_left.set(ship_1_left.get() - 1)  
    if counter == 2:
        ship_2_left.set(ship_2_left.get() - 1)
    if counter == 3:
        ship_3_left.set(ship_3_left.get() - 1)
    if counter == 4:
        ship_4_left.set(ship_4_left.get() - 1)
    logs_update(f'{inp}: Вы потопили {counter}-палубник!')
    logs.itemconfig(END, foreground="green")
    counter = 0


def logs_update(text):      # Добавляет в логи строку
    logs.insert(END, text)
    logs.see(END)


bg_colour = '#d1fffd'  # цвет заднего фона программы
Hint_shown = False  # булевая переменная, отображающая состояние подсказки
counter = 0

dictionary = {'А': 0,
              'Б': 1,
              'В': 2,
              'Г': 3,
              'Д': 4,
              'Е': 5,
              'Ж': 6,
              'З': 7,
              'И': 8,
              'К': 9}

miss_list = ['Промах!', 'Мимо!', 'Вы промахнулись(', 'Пусто!', 'Ничего(']
hit_list = ['Попал!', 'Ранил!', 'В яблочко!', 'Прямо в цель!']  # рандомные фразы для вывода


def ship_generation(length):
    global field
    i, j = rn.randint(1, 10), rn.randint(1, 10)   # координаты для точки
    c1 = []
    vertical = rn.choice([True, False])
    if vertical:
        for x in field[i-1:i+1 + 1]:
            c1.append(x[j-1:j+1+length])
        c2 = [[0 for x in range(j-1, j+1+length)] for x in range(i-1, i+1 + 1)]   # создается двумерный список, с которым будет сверяться c1
        if c1 == c2:
            field[i][j:j+length] = [1 for m in range(length)]
        else:
            return ship_generation(length)
    else:
        for x in field[i - 1:i + 1 + length]:
            c1.append(x[j - 1:j + 1 + 1])
        c2 = [[0 for x in range(j - 1, j + 1 + 1)] for x in range(i - 1, i + 1 + length)]
        if c1 == c2:
            for k in range(length):
                field[i+k][j] = 1
        else:
            return ship_generation(length)


field = [[0 for i in range(12)] for j in range(12)]
for length in [4, 3, 3, 2, 2, 2, 1, 1, 1, 1]:
    ship_generation(length)

del field[0]
del field[-1]
for i in field:
    del i[0]
    del i[-1]
print(field)


# ТУТ НАЧИНАЕТСЯ TKINTER

# создаем окно, куда и будет выводиться игра
root = Tk()
root.title('Морской бой')  # заголовок окна
root.geometry('825x525')  # размер окна
root.resizable(width=False, height=False)  # чтобы нельзя было менять размер окна


# создаем рабочую область, относительно которой будут размещаться остальные объекты
# шаблон: frame_name = Frame(имя родительского виджета/окна, bg='цвет заднего фона')
frame = Frame(root, bg=bg_colour)
# шаблон: widget_name.place(относитаельная ширина, отномительая высота)
frame.place(relwidth=1, relheight=1)

# создаем подпись "МОРСКОЙ БОЙ" вверху окна
title = Label(frame, text='МОРСКОЙ БОЙ', font=('Gabriola', 45), bg=bg_colour)
# шаблон: widget_name.place(ширина, высота, начальная позиция по x, начальная позиция по y)
title.place(width=550, height=50, x=137.5, y=25)

# создаем поле, куда будут вводиться команды
commandInput = Entry(frame)
commandInput.place(width=725, height=25, x=50, y=450)
commandInput.focus_set()  # помещает курсор в поле ввода

# создаем рамку для поля сражения
battlefield_frame = Frame(frame, width=275, height=275, bg=bg_colour)  # width и height можно прописать и тут
battlefield_frame.place(x=275, y=125)
# создаем заголовок из букв
for i in range(25, 250 + 1, 25):
    header_letters = Message(battlefield_frame, text='АБВГДЕЖЗИК'[int(i / 25) - 1], font=25, bg=bg_colour)
    header_letters.place(x=i, y=0, width=25, height=25)
# создаем заголовок из цифр
for j in range(25, 250 + 1, 25):
    header_numbers = Message(battlefield_frame, text=int(j / 25), font=25, bg=bg_colour)
    header_numbers.place(x=0, y=j, width=25, height=25)
# создаем сетку поля сражения
for i in range(25, 250 + 1, 25):
    for j in range(25, 250 + 1, 25):
        draw_cell(i, j, '')

# скрытое поле, чтобы смотреть, где стоят корабли
hidden_battlefield_frame = Frame(frame, width=250, height=250, bg=bg_colour)
for i in range(0, 225 + 1, 25):
    for j in range(0, 225 + 1, 25):
        if field[int(i / 25)][int(j / 25)] == 0:
            color = 'white'
        else:
            color = 'black'
        hidden_cell = Message(hidden_battlefield_frame, relief=RIDGE, bg=color)
        hidden_cell.place(x=i, y=j, width=25, height=25)

# создаем рамку для логов битвы
logs_frame = Frame(frame)
logs_frame.place(x=575, y=150, width=200, height=250)

# создаем полосу прокрутки для логов
scrollbar = Scrollbar(logs_frame, bg=bg_colour, troughcolor=bg_colour)
scrollbar.pack(side='right', fill=Y)

# создаем список, содержащий сами логи
logs = Listbox(logs_frame, relief=FLAT, yscrollcommand=scrollbar.set, height=250, selectbackground='gray')
logs_update('             Добро пожаловать!')
logs.itemconfig(END, foreground="blue")
logs_update('   Для выстрела введите в строку')
logs_update('снизу координаты цели')
logs_update("   Например: 'А1' или 'б2'")
logs_update('   Цель игры - потопить все')
logs_update('корабли противника')
logs_update('   Введите "." для показа поля')
for i in range(1, 7):
    logs.itemconfig(i, foreground="gray")
logs.pack(fill=BOTH)

# привязываем скроллбар к окну с логами
scrollbar.config(command=logs.yview)

# проверка нажатия на Enter
commandInput.bind("<Return>", enter_pressed)

# делаем слева область для отображения количества оставшихся кораблей
ships_left = Frame(frame)
ships_left.place(x=50, y=125, width=200, height=275)
# делаем подпись
ships_left_label = Label(ships_left, text='Кораблей осталось:', font=('Arial', 12), bg=bg_colour)
ships_left_label.place(x=0, y=0, width=200, height=25)
# создаем область, в которой будем рисовать корабли
c = Canvas(ships_left, bg=bg_colour, height=250, highlightthickness=0)
c.pack(side='bottom', fill=BOTH)
# рисуем корабли
for i in range(25, 100 + 1, 25):
    c.create_rectangle(i, 50, i + 25, 75, fill='black', outline=bg_colour)
for i in range(25, 75 + 1, 25):
    c.create_rectangle(i, 100, i + 25, 125, fill='black', outline=bg_colour)
for i in range(25, 50 + 1, 25):
    c.create_rectangle(i, 150, i + 25, 175, fill='black', outline=bg_colour)
c.create_rectangle(25, 200, 50, 225, fill='black', outline=bg_colour)

# создаем переменные для количества оставшихся кораблей
ship_4_left = IntVar()
ship_3_left = IntVar()
ship_2_left = IntVar()
ship_1_left = IntVar()
ship_4_left.set(1)
ship_3_left.set(2)
ship_2_left.set(3)
ship_1_left.set(4)

# создаем тектовые поля с количеством оставшихся кораблей
ship_4 = Message(c, textvariable=ship_4_left, font=15, relief=GROOVE, bg='White')
ship_4.place(x=150, y=50, width=25, height=25)
ship_3 = Message(c, textvariable=ship_3_left, font=15, relief=GROOVE, bg='White')
ship_3.place(x=150, y=100, width=25, height=25)
ship_2 = Message(c, textvariable=ship_2_left, font=15, relief=GROOVE, bg='White')
ship_2.place(x=150, y=150, width=25, height=25)
ship_1 = Message(c, textvariable=ship_1_left, font=15, relief=GROOVE, bg='White')
ship_1.place(x=150, y=200, width=25, height=25)

root.mainloop()


[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [1, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 1, 0, 1, 0, 0, 0], [0, 0, 1, 0, 1, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [1, 1, 0, 1, 0, 1, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 1, 1, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1]]
